## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import os
import numpy as np

# Custom modules
from helpers.change_coordinates import local2global
from helpers.visualization.plots import plot_3d_interactive
from plan import Plan,State
from gcs import GCS
from simulators.QGroundControl.qgc import QGC
from simulators.gazebo.gazebo import Gazebo


#Kill all related process
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Create Plans

In [2]:
offsets = [ #east, north, up, heading
            (5, 5, 0, 90), 
            (10, 0, 0, 45),
            (-5, -10, 0, 225),
            (-15, 0, 0, 0),
            (0, -20, 0, 0),
]
n_vehicles=len(offsets)
side_lens=(5,7,4,1,2) #must agree with n_vehicles
local_paths=[Plan.create_square_path(side_len=side_len,alt= 5) for side_len in side_lens]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5) for path in local_paths]

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'ba

## Gonfiguration

In [3]:
gazebo_word_path="simulators/gazebo/worlds/medical_delivery.world" ##
target_box_center=(-85.263400,56.518300, 0.593695, 0)

origin=(0,0,0)
offsets = [ #east, north, up, heading
    target_box_center, 
    (0,-10, 0, 0),
    (1,-15, 0, 0),
    (2,-10, 0, 0)]

n_vehicles=len(offsets)

homes=np.array([offset[:3] for offset in offsets])

altitudes=[5,5,5,7]
plans= [Plan.hover(alt=alt) for alt in altitudes]

Plan 'hover' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Plan 'hover' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created

## Visualization Parameters

In [4]:
homes=np.array([offset[:3] for offset in offsets])
waypoints = {f'city_homes': {'pos':homes[1:],'color':'blue'},
             f'rural_home': {'pos':homes[[0]],'color':'green'}}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,10],ground=-0.05)

## Choose Simulator

In [5]:
simulator=QGC(offsets=offsets,
              plans=plans,
              origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path=gazebo_word_path,
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [6]:
uavs = simulator.launch()

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'fly' is now connected ✅🔗
Vehicle 1: Action 'fly' is now connected ✅🔗
Vehicle 1: Plan 'hover' is now connected ✅🔗
Vehicle 1: Plan 'hover' is now connected ✅🔗
vehicle 1 created
Vehicle 2: Step 'Check disarmed' is now connected ✅🔗
Vehi

## Execute Plan

In [7]:
gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        i = id-1
        uav=uavs[i]
        uav.plan.act()
        if uav.plan.state in [State.DONE,State.FAILED]:
            to_remove.add(id)
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 3: ▶️ Starting Step: Check disarmed
Vehicle 4: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 3: ✅ Step: Check disarmed is done
Vehicle 4: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 3: ▶️ Starting Step: Check EKF
Vehicle 4: ▶️ Starting Step: Check EKF
Vehicle 4: ✅ Step: Check EKF is done
Vehicle 4: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 4: ✅ Step: Check GPS is done
Vehicle 4: ▶️ Starting Step: Check system
Vehicle 3: ✅ Step: Check EKF is done
Vehicle 3: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 4: ✅ Step: Check system is don

## Kill all related process

In [8]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")